## **Imports**

First, it's necessary to import the libraries that will be used in the script below and the dataset obtained from the [website](https://sistemas.anatel.gov.br/se/public/view/b/licenciamento.php) of the Brazilian National Telecommunications Agency.

### **Libraries**

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
import zipfile
import os

### **Dataset**

The dataset columns used in this scripts are:
* **'NumEstacao':** unique number that characterizes a station;
* **'DataValidade':** expiration date of that radio frequency associated with the station;
* **'Azimute':** positioning in degrees in relation to the north of the antenna radiation main lobe;
* **'AnguloMeiaPotenciaAntena':** half power angle in degrees.

In [2]:
#The variable path must contain a string with the path to the dataset used.
path = '/home/oai-ufrn/Repositories/nir-measurement-methodology/dataset/csv_licenciamento_2e8f645ab8074b48e0421115d284a2da.csv'
df = pd.read_csv(path, encoding='unicode_escape')

## **Data Adjustment**

The cell below creates a dataframe with the stations selected in previous steps of the methodology, it must be changed with the station numbers of the scenario to be evaluated.

In [3]:
filter = {
    'NumStation': [
        922234, 922242, 5187354, 6556752, 431377928, 441068146, 665600062, 686142519,
        686143205, 686696786, 690004206, 690904738, 690905823, 690905866, 690905874,
        690905912, 690905939, 691182450, 691182701, 691182728, 691182760, 691182850,
        691182930, 691182957, 691183031, 691183082, 691223149, 691346259, 691347646,
        691348090, 691571643, 692295992, 692700170, 692817107, 692914501, 693190442,
        695207075, 695364405, 695561146, 695561219, 695706616, 698154673, 698178238,
        1001843867, 1002291736, 1003228795, 1003326657, 1004211527, 1005215631, 1005343001,
        1006733571, 1006918474, 1008016842, 1008016958, 1008246368, 1008246376, 1008690713,
        1008779110, 1009580156, 1010052729, 1015604703, 1015630488
]}
filter = pd.DataFrame(filter)

Some columns are not in the a format usable for the script, so they have been fixed. After this, for each station, is selected the lowest value of half power degree from each unique azimuth value.

In [4]:
#Conversion of the column 'DataValidade' to the datetime64[ns] type.
df['DataValidade'] = pd.to_datetime(df['DataValidade'])

#Exclusion of lines of 'DataValidade' prior to December 31, 2023.
df = df[df['DataValidade'] > pd.to_datetime('2024-01-01', format='%Y-%m-%d')]

#Filtering of the ANATEL's dataset based on the stations manually selected.
df_filtered = df.loc[df['NumEstacao'].isin(filter['NumStation'])].copy()

#Correction of the data type of the columns 'Azimute' and the 'AnguloMeiaPotenciaAntena'.
df_filtered['Azimute'] = pd.to_numeric(df['Azimute'], errors='coerce')
df_filtered['Azimute'] = df_filtered['Azimute'].astype(int)
df_filtered['AnguloMeiaPotenciaAntena'] = pd.to_numeric(df['AnguloMeiaPotenciaAntena'], errors='coerce')

Only the lowest value of the half-power angle must to be used in the plots. The script below arrange the data in a list of tuples, which contain: 
1) The station number;
2) The station azimuths;
3) The smallest half-power angles for each azimuth. 

In [5]:
#Groups the data by 'NumEstacao and 'Azimute' and selects the lowest value of 'AnguloMeiaPotenciaAntena'.
min_values = df_filtered.groupby(['NumEstacao', 'Azimute'])['AnguloMeiaPotenciaAntena'].min().reset_index()

#Finds the minimum non-zero value in 'AnguloMeiaPotenciaAntena'.
for index, row in min_values.iterrows():
    if row['AnguloMeiaPotenciaAntena'] <= 0:
        next_min = df_filtered[(df_filtered['NumEstacao'] == row['NumEstacao']) & (df_filtered['Azimute'] == row['Azimute']) & (df_filtered['AnguloMeiaPotenciaAntena'] > 0)]['AnguloMeiaPotenciaAntena'].min()
        if not math.isnan(next_min):
            min_values.at[index, 'AnguloMeiaPotenciaAntena'] = next_min

#Group by 'NumEstacao' and build the list of azimuths and minimum half-power angle values, forming a list of tuples.
df_stations_info = []
for station in min_values['NumEstacao'].unique():
    temp = min_values[min_values['NumEstacao'] == station]
    azimuths = temp['Azimute'].tolist()
    half_power_angles = temp['AnguloMeiaPotenciaAntena'].tolist()
    df_stations_info.append((station, azimuths, half_power_angles))
#print(df_stations_info)

In case of multi-tenant cell situations, the information of each station must to be grouped, so the function below was designed to do this grouping.

In [6]:
def group_stations(info, *stations):

    # Check if at least two stations are to be grouped
    if len(stations) < 2:
        return "At least two stations are required for merging."

    # Find data for each station in the list
    station_data = []
    for station in stations:
        station_info = None
        for data in info:
            if data[0] == station:
                station_info = data
                break
        if station_info is None:
            return f"Station {station} not found in the configuration list."
        station_data.append(station_info)

    # Combine station data
    new_azimuths = [azimuth for data in station_data if data[1] is not None for azimuth in data[1]]
    new_half_power_angles = [angle for data in station_data if data[2] is not None for angle in data[2]]

    # Create the number of the new station
    new_station_number = ' and '.join(map(str, stations))

    # Remove original stations from the list
    for station in station_data:
        info.remove(station)

    # Add the new combined station to the list
    info.append((new_station_number, new_azimuths, new_half_power_angles))

    # Return the updated list
    return info

The cell below creates a dataframe with the combinations of ERBs that share the same infrastructure. Each column is an association of different Station Numbers, it was also formulated manually following the methodology of the work and needs to be redone for the scenario to be evaluated.

In [7]:
combinations = {
    'station1': [695207075, 1005343001, 1010052729, 1003228795, 1008016842, 1015630488, 691182957],
    'station2': [922234,    690905874,  1009580156, 691183082,  690905866,  1003326657, 692817107],
    'station3': [None,      None,       None,       None,       None,       691182728,  None]
}
multi_tenant_towers = pd.DataFrame(combinations)

#Runs the grouping function
for index, row in multi_tenant_towers.iterrows():
    stations = [int(value) if pd.notnull(value) else None for value in row.values] #Ignore the 'Nones', they only exist so that the list has the same dimensions.
    stations = [station for station in stations if station is not None] #Remove the 'Nones' before creating the list of grouped stations.
    df_stations_grouped_info = group_stations(df_stations_info, *stations)
#print(df_stations_grouped_info)

## **Plots**

The cells below plots the polar graphs of the coverage area of the stations and save it to a .zip file.

In [8]:
def plot(azimuths, half_power_angles, filename):
    #Configures all bars with size 1, causes the visualization of the ERB sector coverage area to be filled from zero to the edge of the graph.
    height_values = [1] * len(azimuths)

    #Convert angles from degrees to radians to be plotted
    azimuths_rad = np.deg2rad(azimuths)
    half_power_angles_rad = np.deg2rad(half_power_angles)

    #Create the polar plot
    plt.figure()
    ax = plt.subplot(111, polar=True)

    #Plot coverage areas
    ax.bar(x=azimuths_rad, height=height_values, width=half_power_angles_rad, color='#FFCE00', edgecolor='#D69400', alpha=0.3)

    #Configure axis limits
    angles = [0, np.pi/2, np.pi, 3*np.pi/2]
    for angle in angles:
        ax.plot([0, angle], [0, 1], color='black')
        plt.gcf().set_facecolor("none")

    #Plot the red azimuth lines
    for azimuth_rad in azimuths_rad:
        ax.plot([0, azimuth_rad], [0, 1], color='red', linewidth='2')
        plt.gcf().set_facecolor("none")

    #Adds 'azimuths_rad' and 'angles'
    angles.extend(azimuths_rad)
    #Replaces '360' values with '0' in the azimuth list so there is no overlap in the image
    for i in range(len(azimuths)):
        if azimuths[i] == 360:
            azimuths[i] = 0
    #Create azimuth labels
    labels = [0, 90, 180, 270]
    labels.extend(azimuths)

    #Plot settings
    ax.set_ylim(0, 1)
    ax.set_xticks(angles)
    ax.set_xticklabels(labels, fontsize=12, weight='bold', fontfamily='serif')
    ax.set_yticklabels([])
    ax.set_theta_direction(-1)
    ax.set_theta_zero_location('N')
    ax.grid(False)
    ax.spines['polar'].set_linewidth(2)

    #Save the figure locally
    plt.savefig(filename, format='png', dpi=600, transparent=True)
    plt.close()

In [9]:
# List to store generated file names
filenames = []

#Generate and save graphs
for station, azimuths, half_power_angles in df_stations_grouped_info:
    filename = f'{station}.png'  # Nome do arquivo único
    plot(azimuths, half_power_angles, filename)
    filenames.append(filename)

#Create and save the .zip file
zip_filename = 'azimuth_analysis.zip'
with zipfile.ZipFile(zip_filename, 'w') as zipf:
    for filename in filenames:
        zipf.write(filename)

#Remove PNG files after adding to zip
for filename in filenames:
    os.remove(filename)